# NLP Tools

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

# Tokenizers

In [0]:
# Tokenizers
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [0]:
# Functions
from pyspark.sql.functions import col, udf

In [0]:
# Types
from pyspark.sql.types import IntegerType

In [0]:
sen_df = spark.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "I wish Java could use case classes"),
    (2, "Logistic,regression,models,are,neat")
], ["id", "sentence"])

In [0]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [0]:
tokenizer = Tokenizer(
    inputCol='sentence',
    outputCol='words'
)

In [0]:
regex_tokenizer = RegexTokenizer(
    inputCol='sentence',
    outputCol='words',
    pattern='\\W'
)

In [0]:
count_tokens = udf(lambda words:len(words, IntegerType()))

In [0]:
tokenized = tokenizer.transform(sen_df)

In [0]:
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish Java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [0]:
tokenized.withColumn('words', count_tokens(col('words'))).show()

---------------------------------------------------------------------------
PythonException                           Traceback (most recent call last)
File <command-3832083736601195>:1
----> 1 tokenized.withColumn('tokens', count_tokens(col('words'))).show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         error_class="NOT_A_BOOLEAN",
    916         message_parameters={"arg_name": "vertical", "arg_type": type(vertical).__name__},
    917     )
    919 if isinstance(truncate, bool) and truncate:
--> 920     pr

In [0]:
rg_tokenized = regex_tokenizer.transform(sen_df)

In [0]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

---------------------------------------------------------------------------
PythonException                           Traceback (most recent call last)
File <command-3832083736601194>:1
----> 1 rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         error_class="NOT_A_BOOLEAN",
    916         message_parameters={"arg_name": "vertical", "arg_type": type(vertical).__name__},
    917     )
    919 if isinstance(truncate, bool) and truncate:
--> 920    

# Stop Words

In [0]:
from pyspark.ml.feature import StopWordsRemover

In [0]:
sentenceDataFrame = spark.createDataFrame([
    (0, ['I', 'saw', 'the', 'green', 'horse']),
    (1, ['Mary', 'had', 'a', 'little', 'lamb'])
], ['id', 'tokens']
)

In [0]:
remover = StopWordsRemover(
    inputCol='tokens',
    outputCol='filtered'
)

In [0]:
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



# n-grams

In [0]:
from pyspark.ml.feature import NGram

In [0]:
wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])

In [0]:
ngram = NGram(n=2, inputCol='words', outputCol='grams')

In [0]:
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



# TF-IDF

In [0]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [0]:
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [0]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [0]:
words_data = tokenizer.transform(sentenceData)

In [0]:
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  0.0|I wish Java could...|[i, wish, java, c...|
|  1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [0]:
hashing_tf = HashingTF(
    inputCol='words',
    outputCol='rawFeatures'
)

In [0]:
featurized_data = hashing_tf.transform(words_data)

In [0]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [0]:
idf_model = idf.fit(featurized_data)

In [0]:
rescaled_data = idf_model.transform(featurized_data)

In [0]:
rescaled_data.select('label', 'features').show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                   |
|0.0  |(262144,[19036,20719,55551,58672,98717,109547,192310],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1.0 

# Count Vectorizer

In [0]:
from pyspark.ml.feature import CountVectorizer

In [0]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [0]:
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2.0)

In [0]:
model = cv.fit(df)

In [0]:
result = model.transform(df)

In [0]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

